In [2]:
conda install tensorflow

Note: you may need to restart the kernel to use updated packages.



The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2022.05=py39_0
  - defaults/win-64::astropy==5.0.4=py39h080aedc_0
  - defaults/win-64::bkcharts==0.2=py39haa95532_0


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\User\anaconda3

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2022.10  |           py39_2          68 KB
    _tflow_select-2.3.0        |              mkl           3 KB
    abseil-cpp-20211102.0      |       hd77b12b_0         1.7 MB
    absl-py-1.3.0              |   py39haa95532_0         171 KB
    anaconda-custom            |           py39_1           4 KB
    astunparse-1.6.3           |             py_0          17 KB
    blinker-1.4                |   py39haa95532_0          23 KB
    ca-certificates-2023.


  - defaults/win-64::bokeh==2.4.2=py39haa95532_0
  - defaults/win-64::bottleneck==1.3.4=py39h080aedc_0
  - defaults/win-64::daal4py==2021.5.0=py39h8cb3d55_0
  - defaults/noarch::dask==2022.2.1=pyhd3eb1b0_0
  - defaults/noarch::datashader==0.13.0=pyhd3eb1b0_1
  - defaults/win-64::datashape==0.5.4=py39haa95532_1
  - defaults/win-64::gensim==4.1.2=py39hd77b12b_0
  - defaults/win-64::h5py==3.6.0=py39h3de5c98_0
  - defaults/noarch::holoviews==1.14.8=pyhd3eb1b0_0
  - defaults/noarch::hvplot==0.7.3=pyhd3eb1b0_1
  - defaults/win-64::imagecodecs==2021.8.26=py39ha1f97ea_0
  - defaults/noarch::imageio==2.9.0=pyhd3eb1b0_0
  - defaults/noarch::intake==0.6.5=pyhd3eb1b0_0
  - defaults/win-64::matplotlib==3.5.1=py39haa95532_1
  - defaults/win-64::matplotlib-base==3.5.1=py39hd77b12b_1
  - defaults/win-64::mkl_fft==1.3.1=py39h277e83a_0
  - defaults/win-64::mkl_random==1.2.2=py39hf11a4ad_0
  - defaults/win-64::numba==0.55.1=py39hf11a4ad_0
  - defaults/win-64::numexpr==2.8.1=py39hb80d3ca_0
  - defaults/w


gast-0.5.3           | 21 KB     |            |   0% 
gast-0.5.3           | 21 KB     | ########## | 100% 
gast-0.5.3           | 21 KB     | ########## | 100% 

_tflow_select-2.3.0  | 3 KB      |            |   0% 
_tflow_select-2.3.0  | 3 KB      | ########## | 100% 
_tflow_select-2.3.0  | 3 KB      | ########## | 100% 

keras-2.9.0          | 1.5 MB    |            |   0% 
keras-2.9.0          | 1.5 MB    | 1          |   1% 
keras-2.9.0          | 1.5 MB    | #2         |  12% 
keras-2.9.0          | 1.5 MB    | ####       |  41% 
keras-2.9.0          | 1.5 MB    | #######9   |  80% 
keras-2.9.0          | 1.5 MB    | ########## | 100% 

keras-preprocessing- | 35 KB     |            |   0% 
keras-preprocessing- | 35 KB     | ########## | 100% 
keras-preprocessing- | 35 KB     | ########## | 100% 

curl-7.84.0          | 142 KB    |            |   0% 
curl-7.84.0          | 142 KB    | #1         |  11% 
curl-7.84.0          | 142 KB    | ########## | 100% 
curl-7.84.0          | 

In [1]:
import tensorflow as tf
from keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from keras.datasets import imdb
from keras.models import Sequential, Model
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [2]:
def transformer_block(embed_dim, num_heads, ff_dim, rate=0.1):
    att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    ffn = Sequential(
        [Dense(ff_dim, activation="relu"),
         Dense(embed_dim), ]
    )
    layernorm1 = LayerNormalization(epsilon=1e-6)
    layernorm2 = LayerNormalization(epsilon=1e-6)
    dropout1 = Dropout(rate)
    dropout2 = Dropout(rate)
    
    def call(inputs, training):
        attn_output = att(inputs, inputs)
        attn_output = dropout1(attn_output, training=training)
        out1 = layernorm1(inputs + attn_output)
        ffn_output = ffn(out1)
        ffn_output = dropout2(ffn_output, training=training)
        return layernorm2(out1 + ffn_output)
    
    return call

In [3]:
def token_and_position_embedding(maxlen, vocab_size, embed_dim):
    token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
    pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)
    
    def call(x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = pos_emb(positions)
        x = token_emb(x)
        return x + positions
    
    return call

In [4]:
import pandas as pd
# Load training data
train_df = pd.read_csv('data/tone_train.csv')
x_train = np.array(train_df['numerical_sequence'].apply(lambda x: [int(i) for i in x.split(',')]).tolist())
y_train = np.array(train_df['BERT_sentiment_score'])

In [5]:
# Load test data
test_df = pd.read_csv('data/tone_test.csv')
x_test = np.array(test_df['numerical_sequence'].apply(lambda x: [int(i) for i in x.split(',')]).tolist())
y_test = np.array(test_df['BERT_sentiment_score'])

# Pad sequences to a fixed length
maxlen = 227 # assuming the maximum sequence length is 200
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, truncating='post')
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen, truncating='post')

In [6]:
print(x_train.shape)
print(y_train.shape)

(12461, 227)
(12461,)


In [7]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# define model architecture
max_seq_length = x_train.shape[1]
vocab_size = np.max(x_train) + 1
embedding_dim = 32
num_heads = 2
ff_dim = 32

inputs = Input(shape=(max_seq_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
transformer_block_fn = transformer_block(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)
transformer_block = transformer_block_fn(embedding_layer, training=True)
pooling_layer = GlobalAveragePooling1D()(transformer_block)
dropout_layer = Dropout(rate=0.1)(pooling_layer)
outputs = Dense(units=5, activation='softmax')(dropout_layer)

model = Model(inputs=inputs, outputs=outputs)

# compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=4, validation_split=0.2)

# save the trained model
model.save('sentiment_analysis_model.h5')

Epoch 1/4
156/156 [==============================] - 26s 140ms/step - loss: 1.4639 - accuracy: 0.3747 - val_loss: 1.3723 - val_accuracy: 0.4489
Epoch 2/4
156/156 [==============================] - 20s 126ms/step - loss: 1.1900 - accuracy: 0.5275 - val_loss: 1.1904 - val_accuracy: 0.5094
Epoch 3/4
156/156 [==============================] - 19s 124ms/step - loss: 0.7783 - accuracy: 0.7061 - val_loss: 1.2518 - val_accuracy: 0.5195
Epoch 4/4
156/156 [==============================] - 19s 122ms/step - loss: 0.3814 - accuracy: 0.8725 - val_loss: 1.5748 - val_accuracy: 0.5158


Do more epochs and get better accuracy
Evaluate model

In [8]:
y_pred = model.predict(x_test)

122/122 [==============================] - 4s 28ms/step


In [9]:
y_pred.shape

(3873, 5)

In [10]:
y_pred[2020]

array([9.9049985e-01, 2.8980316e-03, 1.1658891e-04, 4.9563074e-03,
       1.5292220e-03], dtype=float32)

In [11]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)

# Print the test loss and accuracy
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

122/122 [==============================] - 4s 30ms/step - loss: 1.6645 - accuracy: 0.4901
Test Loss: 1.664543628692627
Test Accuracy: 0.49005937576293945


Have 3 classes instead of 5. Negative, neutral and positive

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Plot training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()